# A3: Project on complex networks
### Analyzing Spotify Data: Global Patterns of Musical Taste and Artist/Genre Clustering by Country
#### _By Saioa Elizondo, Marc Albesa and Maria Fité_
**Group Name:** A3L

**Objective:** Development of a Project in which you put into practice some of the knowledge acquired
during this course on Complex Networks.

**Kind of project:** Analysis. Obtain real data which can be put in terms of network(s),
and analyze the network(s) to answer any question or hypothesis
about them. It is not enough to calculate descriptors or communities,
you must try to answer relevant questions about the system's
functioning.

## 0. Preparing the enviroment

In [ ]:
%pip install spotipy

In [ ]:
# Code to import the needed modules/packages to run the notebook
import os
import pandas as pd # dataframes
import numpy as np # Mathematical operations
import matplotlib.pyplot as plt # Reçpresentation
import random as random
import networkx as nx # Network tools
import spotipy
from scipy.sparse.linalg import eigsh
from spotipy.oauth2 import SpotifyOAuth

# Get workng directory
wd = os.getcwd()

## 1. Importing data and constructing the network
### 1.1. Defining playlist IDs

In [ ]:
playlist_ids = {
    "Catalonia": "5IczcOjRojDGQmphylNgAk",
    "Spain": "7tk0qdAQnySyJC5NIC46LL",
    "Argentina": "7hoGYwZsVAzxSeVDbABqEs",
    "Brazil": "2vUqKjPCtiJN4AL0jVuSiQ",
    "Italy": "0XxpxWDgSajIJubwjSG5p4",
    "France": "4Y9F9p3eOVnAnBdnWXgf3W",
    "USA": "53aDM9BNWGgdbskDrMDBzd",
    "Colombia": "3Oe6dTHiFHtJGu6CgIXvw2",
    "UK": "0ZrNYP9GlN7C89nRcmb0ET",
    "Ecuador": "0gb959SjjZBGj9tsOHpAjU",
    "Mexico": "002EH6cGKQCzSl5XVJhub6",
    "Chile": "7sEuTk63hXvwOyaku7G9oR",
    "Russia": "0UBqxjbJs3nhI2Wexpd0tN",
    "Germany": "0bk2WTv51p9vT5aplcHz3f",
    "Portugal": "4LObto7Z2gRAH4o5H86EqQ",
    "South Korea": "6Ah4okZFJJnDBjUiRhx0Uu",
    "Venezuela": "6BRogt5aan10pDoueAcWgo",
    "Morocco": "2zA1uaGEDg75UdcjZJER4e",
    "Romania": "4jbgA8bDbQSDAjLAFKkTH9",
    "Sweden": "0COXLs3GcoSx80UrlvXUUy",
    "India": "4pbVk8P4zK0cL45qT6PgB1",
    "Uruguay": "61HbuFG0J1Mc7SkEugwzYt",
    "Japan": "65M6poLBwAdYs0TD7qemPk",
    "Ireland": "4nmlpg46HQ1ctO6pbTAI1T",
    "Dominican Republic": "4Krj4NmrOoWIALF3q3yanF",
    "Philippines": "5Ut79yzqJcHoOkUVJIqLWg",
    "Indonesia": "5OazJo0Grq2aP6LsqP8cFP",
    "Austria":"4fxedQjEpyOwCox6Ug5PBi",
    "Peru":"2dxdatUmtaLzJy4lAqPOl0",
    "Australia": "79Mjnq7qvaotdus0eXoYvy",
}

### 1.2. Building the dataset

In [ ]:
# Set up credentials
client_id = '19107210277541c8a31c434e1d5d58c5'
client_secret = 'bf0216ea58b1471a976b1179aec337c5'
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=client_id,
    client_secret=client_secret,
    redirect_uri='http://localhost:8888/callbackif',
    scope="playlist-read-private"
))

# Initialize empty dataframe
dataframe = pd.DataFrame()
genres_by_artist = dict()

# Fetch playlist tracks
for country, playlist_id in playlist_ids.items():
    print(f"Fetching playlist {playlist_id} for {country}")

    # Retrieve playlist items
    try:
        playlist_tracks = sp.playlist_items(playlist_id, market="global")
    except Exception as e:
        print(f"Error fetching playlist {playlist_id}: {e}")
        continue

    # Create and initialize dataframe entry
    country_artists = dict()
    country_artists = {"Country": country}

    for index, item in enumerate(playlist_tracks['items']):
        track = item["track"]

        # Fetch track stats
        Position = index + 1
        Album = track["album"]
        Popularity = track["popularity"]
        Artist = track["artists"][0]['name']
        artist_genres = sp.artist(track["artists"][0]['id']).get('genres', [])

        if Artist not in genres_by_artist.keys():
            genres_by_artist[Artist] = artist_genres

        # Compute popularity score (weight)
        popularity_score = (51-Position)

        # Check if the artist already in the country top50
        if Artist in country_artists.keys():
            country_artists[Artist] += popularity_score
        else: # Add as a new row
            country_artists[Artist] = popularity_score

    # Add the dictionary as a new row
    dataframe = pd.concat([dataframe, pd.DataFrame([country_artists])], ignore_index=True)

# Print DataFrame
dataframe.head()

Fetching playlist 5IczcOjRojDGQmphylNgAk for Catalonia


In [ ]:
# Save DataFrame to a CSV file
dataframe.to_csv(os.path.join(wd, "spotify_global_data.csv"), index=False)  # Set index=False to exclude the index column
print("DataFrame saved to 'spotify_global_data.csv'.")

In [ ]:
# Read the CSV file and load it into a new DataFrame
spotify_global_data = pd.read_csv(os.path.join(wd, "spotify_global_data.csv"))
print("DataFrame loaded from 'spotify_global_data.csv':")

### 1.3. Converting the dataframe to a NetworkX bipartite graph

In [ ]:
# Extract node types
country_nodes = list(spotify_global_data.iloc[:, 0])  # Rows represents countries
artist_nodes = list(spotify_global_data.columns[1:])  # Columns represent artists

# Initialize a bipartite graph
B = nx.Graph()

# Add nodes
B.add_nodes_from(country_nodes, bipartite=0)  # Bipartite group 0
B.add_nodes_from(artist_nodes, bipartite=1)  # Bipartite group 1

# Add edges with weights
for index, row in spotify_global_data.iterrows():
    node_a = row.iloc[0]  # First column (country)
    for node_b in artist_nodes:
        weight = row[node_b]
        if pd.notna(weight):  # Add edge if weight is not NaN
            B.add_edge(node_a, node_b, weight=weight)

# Output results
print("Nodes in the bipartite graph:")
print(B.nodes(data=True))
print("\nEdges in the bipartite graph:")
print(B.edges(data=True))

## 2. Visualizing the network

In [ ]:
# Defining geographical positions (longitude, latitude) for countries
country_positions = {
    "Catalonia": (1.8677, 41.8205),
    "Spain": (-3.7492, 40.4637),
    "Argentina": (-63.6167, -38.4161),
    "Brazil": (-51.9253, -14.2350),
    "Italy": (12.5674, 41.8719),
    "France": (1.8883, 46.6034),
    "USA": (-95.7129, 37.0902),
    "Colombia": (-74.2973, 4.5709),
    "UK": (-3.4360, 55.3781),
    "Ecuador": (-78.1834, -1.8312),
    "Mexico": (-102.5528, 23.6345),
    "Chile": (-71.5430, -35.6751),
    "Russia": (105.3188, 61.5240),
    "Germany": (10.4515, 51.1657),
    "Portugal": (-8.2245, 39.3999),
    "South Korea": (127.7669, 35.9078),
    "Venezuela": (-66.5897, 6.4238),
    "Morocco": (-7.0926, 31.7917),
    "Romania": (24.9668, 45.9432),
    "Sweden": (18.6435, 60.1282),
    "India": (78.9629, 20.5937),
    "Uruguay": (-55.7658, -32.5228),
    "Japan": (138.2529, 36.2048),
    "Ireland": (-7.6921, 53.1424),
    "Dominican Republic": (-70.1627, 18.7357),
    "Philippines": (121.7740, 12.8797),
    "Indonesia": (113.9213, -0.7893),
    "Austria": (14.5501, 47.5162),
    "Peru": (-75.0152, -9.1900),
    "Australia": (133.7751, -25.2744),
}

In [ ]:
# Helper function to separate overlapping nodes
def repel_nodes(positions, min_dist=0.1):
    for node1, pos1 in positions.items():
        for node2, pos2 in positions.items():
            if node1 != node2:
                dist = np.linalg.norm(np.array(pos1) - np.array(pos2))
                if dist < min_dist:  # If nodes are too close
                    direction = np.array(pos1) - np.array(pos2)
                    direction = direction / np.linalg.norm(direction) if np.linalg.norm(direction) != 0 else np.random.rand(2)
                    positions[node1] += direction * (min_dist - dist) * 0.5
                    positions[node2] -= direction * (min_dist - dist) * 0.5
    return positions

In [ ]:
# Generate force-directed layout for artist nodes
country_nodes = [node for node, data in B.nodes(data=True) if data.get("bipartite") == 0]
country_labels = {node: node for node in country_nodes}
artist_nodes = [node for node, data in B.nodes(data=True) if data.get("bipartite") == 1]
artist_labels = {node: node for node in artist_nodes}

# Compute positions
fixed_positions = {node: pos for node, pos in country_positions.items() if node in country_nodes}
# Spring layout for artist nodes
positions = nx.spring_layout(B, pos=fixed_positions, fixed=country_nodes, weight="weight", iterations=50,k=5)

# Apply repulsion to avoid overlapping nodes
artist_positions = {node: positions[node] for node in artist_nodes}
artist_positions = repel_nodes(artist_positions, min_dist=10)

# Combine back the positions, keeping country nodes fixed
for node in country_nodes:
    positions[node] = fixed_positions[node]
for node in artist_nodes:
    positions[node] = artist_positions[node]

# Separate positions for country and artist nodes
country_positions = {node: positions[node] for node in country_nodes}
artist_positions = {node: positions[node] for node in artist_nodes}

# Plotting
plt.figure(figsize=(14, 12))
edge_widths = [((data.get("weight", 1) - 1)/531)*(10)+1 for _, _, data in B.edges(data=True)]

# Draw nodes
nx.draw_networkx_nodes(B, positions, nodelist=artist_nodes, node_size=50, node_color="lightblue", label="Artists")
nx.draw_networkx_nodes(B, positions, nodelist=country_nodes, node_size=250, node_color="plum", label="Countries")
nx.draw_networkx_labels(B, country_positions, labels=country_labels, font_size=8, font_color="black")

# Draw edges
nx.draw_networkx_edges(B, positions, edge_color="gray", alpha=0.3, width=edge_widths)

plt.legend()
plt.gca().set_frame_on(False)
plt.show()

## 3. Regional claustering of top artists

The main goal of this section is to explore whether countries within the same geographical region tend to have a more similar musical taste.

First of all we need to create a bipartite network with two types of nodes. One type corresponds to the different countries and the other type to the name of the artists. Each artist is linked to a country if it appears in the playlist of a given country. Once done that, as we are interested only in countries, we will project the bipartite network onto a unipartite graph of countries. This will give rise to a network consisting in the different countries as nodes, where weights are automatically computed as the number of common neighbors (artists).

In [ ]:
# Extract node types
country_nodes = list(spotify_global_data.iloc[:, 0])  # Rows represents countries
artist_nodes = list(spotify_global_data.columns[1:])  # Columns represent artists

# Create a bipartite graph
G = nx.Graph()
country_nodes = list(spotify_global_data.iloc[:, 0])  # Rows represents countries
artist_nodes = list(spotify_global_data.columns[1:])  # Columns represent artists
G.add_nodes_from(country_nodes, bipartite=0)
G.add_nodes_from(artist_nodes, bipartite=1)
for index, row in spotify_global_data.iterrows():
    node_a = row.iloc[0]  # First column (country)
    for node_b in artist_nodes:
        weight = row[node_b]
        if pd.notna(weight):  # Add edge if weight is not NaN
            G.add_edge(node_a, node_b, weight=1)

# Project the graph onto the set of countries
country_graph = nx.bipartite.weighted_projected_graph(G, country_nodes)

# Display graph
plt.figure(figsize=(12, 12))
pos = nx.spring_layout(country_graph, seed=42)
nx.draw_networkx_nodes(country_graph, country_positions, node_color="plum", node_size=300, alpha=0.8)
nx.draw_networkx_edges(country_graph, positions, edge_color="gray", alpha=0.5)
nx.draw_networkx_labels(country_graph, country_positions, font_size=10, font_color="black")
plt.title("Network with countries", fontsize=14)
plt.axis('off')
plt.show()

Once having the network, we will apply a Louvian community detection algorithm in order to detect comunities acording to the similarity in artists among different countries.

In [ ]:
import community.community_louvain as community_louvain

# Compute the best partition
partition = community_louvain.best_partition(country_graph, weight='weight')

# Add communities to node attributes
nx.set_node_attributes(country_graph, partition, 'community')

Now we will plot the Network with the countries where edge thickness is proportional to similarity and node color indicates the community membership.

In [ ]:
import matplotlib.cm as cm

# Each community gets a unique color
communities = set(partition.values())
color_map = cm.get_cmap('tab20', len(communities))
colors = {community: color_map(i) for i, community in enumerate(communities)}

# Make plot
plt.figure(figsize=(12, 12))

# Get node colors
node_colors = [colors[partition[node]] for node in country_graph.nodes()]

# Draw the graph
nx.draw_networkx_nodes(country_graph, country_positions, node_color=node_colors, node_size=700, alpha=0.8)
nx.draw_networkx_edges(country_graph, positions, edge_color="gray", alpha=0.5)
nx.draw_networkx_labels(country_graph, country_positions, font_size=10, font_color="black")

# Add legend for communities
for comm_id, color in colors.items():
    plt.scatter([], [], color=color, label=f'Community {comm_id}', s=100)
plt.legend(scatterpoints=1, frameon=True, title="Communities", loc='best')

# Add title and show plot
plt.title("Community Detection for the Different Countries", fontsize=14)
plt.axis('off')
plt.show()


Additionally, we will also print a dictionary where keys are the different comunities and the values inside the key the countries belonging to that comunity.

In [ ]:
from collections import defaultdict

communities = defaultdict(list)
for country, comm_id in partition.items():
    communities[comm_id].append(country)
for comm_id, countries in communities.items():
    print(f"Comunity {comm_id}: {', '.join(map(str, countries))}")

Our algorithm has been capable of detecting 5 different comunities. First of all we can see that Russia and Catalonia are alone in their own comunity. This fact might be due to that they listent to local artists and they are not much related to other countries regarding to musical taste. Additionally, we can see that Latin America and Spain are detected to be an other different comunity, this makes sense if we consider the language spoken in those countries, as they might listen to Spanish-speaker artists. Moreover, asiatic countries seem to be grouped in another comunity, as they might support artist that are from the continent. Finally, European countries, Australia, USA and Brasil seem to shear the same comunity. This fact could be due to the intense relationship in culture and the globalization that makes all those countries close to each other regarding to music taste.

## 4. Genre Diversity by Country

The main goal of thsi section is to study countries with a wider variety of most-listened genres and relate to the cultural diversity of each country.

First of all we need to create a bipartite network with two types of nodes. One type corresponds to the different countries and the other type the genere. Each genere is linked to a country if it appears in the playlist of a given country. To calculate the weights, the genere of the artist is checked. If it is asigned to more than one genere a weight of one is devided into the total number of generes, contrary it is assigned to 1. The total weight of a link between genere and country is the result of the sum of the weight contribution of each song.

In [ ]:

track_id = "3n3Ppam7vgaVa1iaRUc9Lp"  # Example: "Mr. Brightside" by The Killers

# Get track details
track = sp.track(track_id)
artists = track['artists']  # Get the list of artists for the track

# Fetch genres for each artist
for artist in artists:
    artist_id = artist['id']
    artist_info = sp.artist(artist_id)  # Get artist details
    genres = artist_info.get('genres', [])  # Get genres
    print(f"Artist: {artist['name']}")
    print(f"Genres: {', '.join(genres) if genres else 'No genres available'}")

In [ ]:
# Create dictionary with generes for each artist
dictionary_generes = {
    'Alfred García': ['Pop', 'Jazz', 'Alternative Rock'],
    'Oques Grasses': ['Pop', 'Rock', 'Funk'],
    'Figa Flawas': ['Hip-Hop', 'Reggae'],
    "Lal'Ba": ['Pop', 'Hip-Hop'],
    'Els Amics De Les Arts': ['Pop', 'Indie Rock'],
    'JULS': ['Reggaeton', 'Trap'],
    'Miki Núñez': ['Pop', 'Rock', 'Indie'],
    'Sexenni': ['Pop', 'Indie'],
    'Buhos': ['Pop', 'Rock'],
    'Ginestà': ['Pop', 'Folk'],
    'Suu': ['Pop', 'Indie Pop'],
    'Will.X.O': ['Trap', 'Rap'],
    'Adrien Broadway': ['Pop', 'R&B'],
    'La Fúmiga': ['Pop', 'Reggae', 'Ska'],
    'Joan Dausà': ['Pop', 'Indie'],
    'Gigi Ros': ['Pop', 'Funk'],
    'Mushkaa': ['Hip-Hop', 'Trap'],
    'Flashy Ice Cream': ['Hip-Hop', 'Trap'],
    'Sopa De Cabra': ['Rock', 'Pop Rock'],
    'Mama Dousha': ['Hip-Hop', 'Rap'],
    'dani6ix': ['Trap', 'Rap'],
    'CLASSE B': ['Hip-Hop', 'Rap'],
    'Laura Gibert': ['Pop', 'Indie'],
    'The Tyets': ['Pop', 'Indie'],
    "Ven'nus": ['Pop', 'Reggae'],
    'Galgo Lento': ['Pop', 'Indie'],
    'Scorpio': ['Pop', 'Trap'],
    'Julieta': ['Pop', 'Folk'],
    'xicu': ['Pop', 'Rock'],
    'Bali 13': ['Trap', 'Hip-Hop'],
    'Júlia Blum': ['Pop', 'Indie'],
    'Miquel Abras': ['Pop', 'Rock'],
    'Sara Roy': ['Pop', 'Indie'],
    'Itaca Band': ['Folk', 'Pop'],
    'Iglú': ['Pop', 'Indie'],
    'Yung Rajola': ['Trap', 'Rap'],
    'Uri Santafé': ['Pop', 'Indie'],
    'Al·lèrgiques al pol·len': ['Pop', 'Indie'],
    'Beatneeks': ['Hip-Hop', 'Rap'],
    'Lluís Sánchez': ['Pop', 'Folk'],
    'BALMA': ['Pop', 'Indie'],
    'Siderland': ['Rock', 'Pop'],
    'La Tour De Carol': ['Pop', 'Folk'],
    'Blaumut': ['Pop', 'Folk'],
    'Alleh': ['Pop', 'Trap'],
    'Quevedo': ['Reggaeton', 'Trap'],
    'Danny Ocean': ['Reggaeton', 'Pop'],
    'Myke Towers': ['Trap', 'Reggaeton'],
    'Cano': ['Rap', 'Trap'],
    'Clarent': ['Pop', 'Reggae'],
    'KAROL G': ['Reggaeton', 'Pop'],
    'Yan Block': ['Rap', 'Hip-Hop'],
    'Beéle': ['Reggaeton', 'Pop'],
    'Blessd': ['Reggaeton', 'Trap'],
    'Moncho Chavea': ['Reggaeton', 'Rap'],
    'Bizarrap': ['Trap', 'Rap'],
    'JC Reyes': ['Reggaeton', 'Pop'],
    'Rels B': ['Trap', 'Hip-Hop'],
    'Omar Courtz': ['Trap', 'Rap'],
    'Rauw Alejandro': ['Reggaeton', 'Pop'],
    'Los Sufridos': ['Rock', 'Pop'],
    'Rvfv': ['Rap', 'Trap'],
    'ROSÉ': ['Pop', 'K-Pop'],
    'Bad Bunny': ['Reggaeton', 'Trap'],
    'Beny Jr': ['Reggaeton', 'Trap'],
    'Lady Gaga': ['Pop', 'Dance'],
    'Feid': ['Reggaeton', 'Pop'],
    'ROA': ['Rap', 'Trap'],
    'Lola Indigo': ['Pop', 'Reggaeton'],
    'Kapo': ['Trap', 'Rap'],
    'FloyyMenor': ['Trap', 'Rap'],
    'La La Love You': ['Pop', 'Indie'],
    'Gonzy': ['Trap', 'Rap'],
    'Amaia': ['Pop', 'Indie'],
    'Morad': ['Trap', 'Rap'],
    'Dani Fernández': ['Pop', 'Indie'],
    'DELLAFUENTE': ['Pop', 'Rap'],
    'Duki': ['Trap', 'Rap'],
    'Sebastian Yatra': ['Pop', 'Reggaeton'],
    'Dei V': ['Rap', 'Trap'],
    'Cyril Kamer': ['Rap', 'R&B'],
    'Gracie Abrams': ['Pop', 'Indie'],
    'Shakira': ['Pop', 'Latin'],
    'La T y La M': ['Pop', 'Reggaeton'],
    'Gordo': ['Rap', 'Trap'],
    'Jombriel': ['Rap', 'Trap'],
    'Tiago PZK': ['Trap', 'Rap'],
    'Renzo ED': ['Rap', 'Trap'],
    'Big One': ['Trap', 'Rap'],
    'Los Ángeles Azules': ['Cumbia', 'Latin'],
    'Luck Ra': ['Trap', 'Rap'],
    'Salastkbron': ['Trap', 'Hip-Hop'],
    'Valentino Merlo': ['Pop', 'Rap'],
    'Emilia': ['Pop', 'Latin'],
    'Pinky SD': ['Trap', 'Rap'],
    "Q' Lokura": ['Reggaeton', 'Trap'],
    'Emanero': ['Rap', 'Trap'],
    'TINI': ['Pop', 'Latin'],
    'ECKO': ['Trap', 'Rap'],
    'King Savagge': ['Rap', 'Trap'],
    'Yami Safdie': ['Rap', 'Trap'],
    "Hernan y La Champion's Liga": ['Rap', 'Trap'],
    'Paulo Londra': ['Trap', 'Rap'],
    'No Te Va Gustar': ['Rock', 'Latin'],
    'Nicki Nicole': ['Trap', 'Pop'],
    'Standly': ['Rap', 'Trap'],
    'DJ Tao': ['Dance', 'Electronic'],
    'Babasonicos': ['Rock', 'Alternative Rock'],
    'Lucky Brown': ['Jazz', 'Soul'],
    'Oruam': ['Pop', 'Rap'],
    'Nilo': ['Reggaeton', 'Pop'],
    'Henry Freitas': ['Rap', 'Trap'],
    'Brenno & Matheus': ['Pop', 'Rap'],
    'Eric Land': ['Pop', 'R&B'],
    'MC Tuto': ['Trap', 'Rap'],
    'Grupo Menos É Mais': ['Samba', 'Latin'],
    'Nuzio Medeiros': ['Trap', 'Rap'],
    'Léo Foguete': ['Trap', 'Rap'],
    'J. Eskine': ['Trap', 'Rap'],
    'NATTAN': ['Trap', 'Rap'],
    "Clayton & Romário": ["Sertanejo"],
    "Jorge & Mateus": ["Sertanejo"],
    "MC LUUKY": ["Rap", "Hip-Hop"],
    "Hugo & Guilherme": ["Sertanejo"],
    "Henrique & Juliano": ["Sertanejo"],
    "Guilherme & Benuto": ["Sertanejo"],
    "DJ TOPO": ["Electronic"],
    "Mc Negão Original": ["Funk"],
    "Murilo Huff": ["Sertanejo"],
    "Maiara & Maraisa": ["Sertanejo"],
    "MC Marks": ["Funk"],
    "MC Cebezinho": ["Funk"],
    "Luan Santana": ["Sertanejo"],
    "Gusttavo Lima": ["Sertanejo"],
    "Luan Pereira": ["Sertanejo"],
    "Ferrugem": ["Pagode", "Samba"],
    "Israel & Rodolffo": ["Sertanejo"],
    "DJ Douglinhas": ["Electronic"],
    "Felipe Araújo": ["Sertanejo"],
    "Wesley Safadão": ["Forró"],
    "Bruno & Denner": ["Sertanejo"],
    "Thiago Carvalho": ["Pop", "Sertanejo"],
    "Supernova Ent": ["Electronic"],
    "Kaique e Felipe": ["Sertanejo"],
    "Grelo": ["Funk"],
    "Matheus & Kauan": ["Sertanejo"],
    "PEDRO SAMPAIO": ["Funk", "Electronic"],
    "Boladin 211": ["Funk"],
    "MC GP": ["Funk"],
    "MC Kevinho": ["Funk"],
    "Marracash": ["Rap", "Hip-Hop"],
    "Simba La Rue": ["Rap", "Hip-Hop"],
    "Artie 5ive": ["Electronic"],
    "Pinguini Tattici Nucleari": ["Indie", "Pop Rock"],
    "Olly": ["Pop", "Indie"],
    "Cesare Cremonini": ["Pop"],
    "Alfa": ["Pop"],
    "Baby Gang": ["Rap", "Hip-Hop"],
    "Papa V": ["Rap", "Hip-Hop"],
    "Geolier": ["Rap", "Hip-Hop"],
    "ANNA": ["Electronic"],
    "Achille Lauro": ["Rap", "Pop"],
    "Ghali": ["Rap", "Hip-Hop"],
    "Sal Da Vinci": ["Pop", "Italian"],
    "Lazza": ["Rap", "Hip-Hop"],
    "Tony Effe": ["Rap", "Hip-Hop"],
    "Lele Blade": ["Rap", "Hip-Hop"],
    "Damiano David": ["Rock", "Pop"],
    "CoCo": ["Pop"],
    "RRARI DAL TACCO": ["Rap", "Hip-Hop"],
    "Kid Yugi": ["Rap", "Hip-Hop"],
    "Astro": ["Pop"],
    "Ornella Vanoni": ["Italian Pop"],
    "Capo Plaza": ["Rap", "Hip-Hop"],
    "Ultimo": ["Pop"],
    "Billie Eilish": ["Pop"],
    "JVLI": ["Pop"],
    "Gazo": ["Rap", "Hip-Hop"],
    "GIMS": ["Pop", "Rap"],
    "Damso": ["Rap", "Hip-Hop"],
    "SDM": ["Rap", "Hip-Hop"],
    "Joé Dwèt Filé": ["Rap", "Hip-Hop"],
    "Bouss": ["Rap", "Hip-Hop"],
    "Dr. Yaro": ["Rap", "Hip-Hop"],
    "Arcane": ["Electronic"],
    "Theodora": ["Pop"],
    "Jul": ["Rap", "Hip-Hop"],
    "Guy2Bezbar": ["Rap", "Hip-Hop"],
    "SANTA": ["Rap", "Hip-Hop"],
    "Werenoi": ["Pop"],
    "Favé": ["Rap", "Hip-Hop"],
    "Ninho": ["Rap", "Hip-Hop"],
    "Sevdaliza": ["Alternative", "Electronic"],
    "Teddy Swims": ["R&B"],
    "Lola Young": ["Soul", "R&B"],
    "Linh": ["Pop"],
    "AMK": ["Pop"],
    "La Mano 1.9": ["Rap", "Hip-Hop"],
    "Helena": ["Pop"],
    "Tiakola": ["Rap", "Hip-Hop"],
    "Kavinsky": ["Electronic"],
    "Booba": ["Rap", "Hip-Hop"],
    "Denden": ["Rap", "Hip-Hop"],
    "Soolking": ["Rap", "Hip-Hop"],
    "Macklemore": ["Rap", "Hip-Hop"],
    "Alex Warren": ["Pop"],
    "David Guetta": ["Electronic"],
    "Benson Boone": ["Pop"],
    "Sabrina Carpenter": ["Pop"],
    "VEN1": ["Pop"],
    "Jaden Bojsen": ["Pop"],
    "Kendrick Lamar": ["Rap", "Hip-Hop"],
    "Tyler, The Creator": ["Rap", "Hip-Hop"],
    "Morgan Wallen": ["Country"],
    "Gigi Perez": ["Pop"],
    "The Weeknd": ["Pop"],
    "Chappell Roan": ["Pop"],
    "Jimin": ["Pop"],
    "SZA": ["R&B"],
    "Post Malone": ["Pop", "Rap"],
    "Lil Baby": ["Rap", "Hip-Hop"],
    "Shaboozey": ["Rap", "Hip-Hop"],
    "Don Toliver": ["Rap", "Hip-Hop"],
    "The Marías": ["Indie Pop"],
    "GloRilla": ["Rap", "Hip-Hop"],
    "The Neighbourhood": ["Indie Rock"],
    "Noah Kahan": ["Folk", "Pop"],
    "Doechii": ["Rap", "Hip-Hop"],
    "Tommy Richman": ["Pop"],
    "Hozier": ["Indie Rock", "Soul"],
    "Neton Vega": ["Pop"],
    "Imogen Heap": ["Electronic", "Indie"],
    "Sam Barber": ["Pop"],
    "Zach Bryan": ["Country"],
    "W Sound": ["Pop"],
    "Maisak": ["Rap", "Hip-Hop"],
    "Heredero": ["Reggaeton"],
    "Nanpa Básico": ["Rap", "Hip-Hop"],
    "Kris R.": ["Rap", "Hip-Hop"],
    "Ryan Castro": ["Reggaeton"],
    "Maluma": ["Reggaeton"],
    "ONIKX": ["Rap", "Hip-Hop"],
    "Reykon": ["Reggaeton"],
    "Andy Rivera": ["Reggaeton"],
    "Grupo Niche": ["Salsa"],
    "Oscar Maydon": ["Reggaeton"],
    "J Balvin": ["Reggaeton"],
    "Boza": ["Reggaeton"],
    "Dalmata": ["Reggaeton"],
    "BTS": ["K-Pop"],
    "Frankie Ruiz": ["Salsa"],
    "Kenny Die": ["Pop"],
    "Myles Smith": ["Pop"],
    "The Killers": ["Rock"],
    "Coldplay": ["Rock"],
    "Chrystal": ["Pop"],
    "Dream Supplier": ["Electronic"],
    "BL3SS": ["R&B"],
    "Cynthia Erivo": ["R&B", "Soul"],
    "Sonny Fodera": ["Electronic"],
    "Tyla": ["R&B"],
    "Fleetwood Mac": ["Rock"],
    "Naughty Boy": ["Electronic"],
    "The Police": ["Rock"],
    "Dasha": ["Pop"],
    "PAWSA": ["Electronic"],
    "Kings of Leon": ["Rock"],
    "DJ Lalo": ["Electronic"],
    "Plan B": ["Reggaeton"],
    "Yailin la Mas Viral": ["Reggaeton"],
    "Sossa": ["Rap", "Hip-Hop"],
    "Leo Dan": ["Pop"],
    "Lalo Rodriguez": ["Salsa"],
    "Rvssian": ["Reggaeton"],
    "Adolescent's Orquesta": ["Salsa"],
    "Grupo Frontera": ["Banda"],
    "Tito Double P": ["Rap", "Hip-Hop"],
    "Gabito Ballesteros": ["Reggaeton"],
    "Peso Pluma": ["Banda"],
    "Fuerza Regida": ["Banda"],
    "Jorsshh": ["Rap", "Hip-Hop"],
    "Chino Pacas": ["Banda"],
    "Natanael Cano": ["Banda", "Corridos"],
    "Junior H": ["Banda"],
    "Clave Especial": ["Salsa"],
    "Carin Leon": ["Banda"],
    "Luis R Conriquez": ["Banda"],
    "Victor Mendivil": ["Banda"],
    "Julión Álvarez y su Norteño Banda": ["Banda"],
    "Victor Valverde": ["Banda"],
    "LEGADO 7": ["Banda"],
    "Cris Mj": ["Rap", "Hip-Hop"],
    "Martinwhite": ["Rap", "Hip-Hop"],
    "Jere Klein": ["Pop"],
    "Grupo Zúmbale Primo": ["Salsa"],
    "Bayron Fire": ["Banda"],
    "Fran C": ["Banda"],
    "Santaferia": ["Cumbia"],
    "Ovy On The Drums": ["Reggaeton"],
    "Jairo Vera": ["Salsa"],
    "Gino Mella": ["Salsa"],
    "SpuTniK Project": ["Electronic"],
    "Liaze": ["Pop"],
    "ANNA ASTI": ["Pop"],
    "Jakone": ["Pop"],
    "Irina Dubtsova": ["Pop"],
    "wagna": ["Pop"],
    "MACAN": ["Pop"],
    "Konfuz": ["Rap", "Hip-Hop"],
    "Geegun": ["Pop"],
    "Big Baby Tape": ["Rap", "Hip-Hop"],
    "Bearwolf": ["Rap", "Hip-Hop"],
    "Gio Pika": ["Pop"],
    "BLIZKEY": ["Rap", "Hip-Hop"],
    "BEARWOLF": ["Rap", "Hip-Hop"],
    "Poshlaya Molly": ["Pop", "Rock"],
    "Татьяна Куртукова": ["Pop"],
    "Aarne": ["Pop"],
    "Xholidayboy": ["Rap", "Hip-Hop"],
    "Женя Трофимов": ["Pop"],
    "Navai": ["Rap", "Hip-Hop"],
    "6YNTHMANE": ["Rap", "Hip-Hop"],
    "Wallem": ["Pop"],
    "Leonid Agutin": ["Pop"],
    "Misha Xramovi": ["Pop"],
    "Xcho": ["Pop"],
    "Irina Kairatovna": ["Pop"],
    "KARAT": ["Pop"],
    "UNIK": ["Pop"],
    "VEIGEL": ["Pop"],
    "Криминальный бит": ["Rap", "Hip-Hop"],
    "Merab Amzoevi": ["Pop"],
    "Вектор А": ["Pop"],
    "Bittuev": ["Pop"],
    "WhyBaby?": ["Rap", "Hip-Hop"],
    "A.V.G": ["Rap", "Hip-Hop"],
    "Mult96": ["Rap", "Hip-Hop"],
    "TIGO": ["Rap", "Hip-Hop"],
    "Costa Lacoste": ["Pop"],
    "Lusia Chebotina": ["Pop"],
    "ebert": ["Pop"],
    "Linkin Park": ["Rock"],
    "Nina Chuba": ["Pop"],
    "Sosa La M": ["Rap", "Hip-Hop"],
    "Jazeek": ["Pop"],
    "LACAZETTE": ["Pop"],
    "Bonez MC": ["Rap", "Hip-Hop"],
    "SAMIRA": ["Pop"],
    "Loredana": ["Pop"],
    "AYLIVA": ["Pop"],
    "Shirin David": ["Pop"],
    "Berq": ["Pop"],
    "Artemas": ["Pop"],
    "FOURTY": ["Pop"],
    "HUGEL": ["Electronic"],
    "Dimitri Vegas & Like Mike": ["Electronic"],
    "cassö": ["Electronic"],
    "Journey": ["Rock"],
    "Bobby Vandamme": ["Electronic"],
    "Natasha Bedingfield": ["Pop"],
    "CYRIL": ["Pop"],
    "Mark Ambor": ["Pop"],
    "Aymen": ["Pop"],
    "Pashanim": ["Rap", "Hip-Hop"],
    "Souly": ["Pop"],
    "Udo Lindenberg": ["Rock"],
    "Plutonio": ["Rap", "Hip-Hop"],
    "Mizzy Miles": ["Rap", "Hip-Hop"],
    "Dillaz": ["Rap", "Hip-Hop"],
    "Slow J": ["Rap", "Hip-Hop"],
    "Calema": ["Pop"],
    "Bluay": ["Pop"],
    "Mc Livinho": ["Funk"],
    "Adam Port": ["Electronic"],
    "Richie Campbell": ["Reggae"],
    "Matuê": ["Rap", "Hip-Hop"],
    "Jung Kook": ["K-Pop"],
    "V": ["K-Pop"],
    "Jin": ["K-Pop"],
    "aespa": ["K-Pop"],
    "G-DRAGON": ["K-Pop"],
    "DAY6": ["K-Pop"],
    "BABYMONSTER": ["K-Pop"],
    "BIGBANG": ["K-Pop"],
    "ILLIT": ["K-Pop"],
    "NewJeans": ["K-Pop"],
    "fromis_9": ["K-Pop"],
    "Lim Young Woong": ["K-Pop"],
    "KISS OF LIFE": ["K-Pop"],
    "HAON": ["K-Pop"],
    "WOODZ": ["K-Pop"],
    "SIXTONES": ["K-Pop"],
    "BTS": ["K-Pop"],
    "MOMOLAND": ["K-Pop"],
    "Axel Rulay": ["Rap", "Hip-Hop"],
    "Rawayana": ["Reggae", "Pop"],
    "ElGrandeToto": ["Rap", "Hip-Hop"],
    "La Fouine": ["Rap", "Hip-Hop"],
    "Figoshin": ["Rap", "Hip-Hop"],
    "Lazaro": ["Reggae", "Dancehall"],
    "Pause": ["Pop"],
    "Tawsen": ["Pop"],
    "Draganov": ["Rap", "Hip-Hop"],
    "Najm": ["Pop"],
    "Delarue": ["Rap", "Hip-Hop"],
    "Duke": ["Rap", "Hip-Hop"],
    "Bilel Tacchini": ["Pop"],
    "KALIL": ["Rap", "Hip-Hop"],
    "Bo9al": ["Rap", "Hip-Hop"],
    "SKY": ["Rap", "Hip-Hop"],
    "Ali S": ["Rap", "Hip-Hop"],
    "Stormy": ["Rap", "Hip-Hop"],
    "Douaa Lahyaoui": ["Pop", "Arabic"],
    "Niro": ["Rap", "Hip-Hop"],
    "Snor": ["Rap", "Hip-Hop"],
    "LFERDA": ["Rap", "Hip-Hop"],
    "Mons": ["Rap", "Hip-Hop"],
    "Nouamane Belaiachi": ["Pop", "Arabic"],
    "L'morphine": ["Rap", "Hip-Hop"],
    "Lbenj": ["Rap", "Hip-Hop"],
    "Theo Rose": ["Pop", "Romanian"],
    "Dani Mocanu": ["Pop", "Romanian"],
    "Tzanca Uraganu": ["Pop", "Romanian"],
    "Iuly Neamtu": ["Pop", "Romanian"],
    "Babasha": ["Pop", "Arabic"],
    "Sami G": ["Pop", "Arabic"],
    "AlbertNbn": ["Rap", "Hip-Hop"],
    "Gya": ["Pop"],
    "NOUA UNSPE": ["Pop"],
    "Denis Ramniceanu": ["Pop", "Romanian"],
    "candyboii": ["Rap", "Hip-Hop"],
    "IDK": ["Rap", "Hip-Hop"],
    "Mgk666": ["Rap", "Hip-Hop"],
    "Raihold": ["Pop"],
    "RAVA": ["Rap", "Hip-Hop"],
    "Bogdan DLP": ["Rap", "Hip-Hop"],
    "YNY Sebi": ["Rap", "Hip-Hop"],
    "Alex Botea": ["Pop"],
    "Oscar": ["Pop"],
    "Costel Biju": ["Pop", "Romanian"],
    "Jador": ["Pop", "Romanian"],
    "Bolaget": ["Pop"],
    "Humlan Djojj": ["Pop"],
    "Babblarna": ["Pop", "Children"],
    "Gaboro": ["Pop"],
    "Molly Sandén": ["Pop"],
    "Yasin": ["Rap", "Hip-Hop"],
    "Soppgirobygget": ["Pop", "Indie"],
    "Y4ska": ["Pop", "Rap"],
    "Avicii": ["Electronic", "Dance"],
    "ADAAM": ["Pop"],
    "Asme": ["Pop"],
    "Miriam Bryant": ["Pop"],
    "Purple Disco Machine": ["Electronic", "Dance"],
    "Miss Li": ["Pop"],
    "Sarettii": ["Pop"],
    "Lord Huron": ["Indie Rock", "Folk"],
    "Hooja": ["Electronic"],
    "Anurag Saikia": ["Bollywood", "Soundtrack"],
    "Anuv Jain": ["Pop", "Indie"],
    "Jasleen Royal": ["Pop", "Indie"],
    "Sachet-Parampara": ["Bollywood", "Soundtrack"],
    "Faheem Abdullah": ["Pop", "R&B"],
    "Karan Aujla": ["Rap", "Punjabi"],
    "Yo Yo Honey Singh": ["Rap", "Punjabi"],
    "Ram Sampath": ["Bollywood", "Soundtrack"],
    "Sachin-Jigar": ["Bollywood", "Soundtrack"],
    "Aditya Rikhari": ["Bollywood", "Soundtrack"],
    "Arijit Singh": ["Bollywood", "Pop"],
    "Dhanda Nyoliwala": ["Punjabi", "Pop"],
    "Pritam": ["Bollywood", "Soundtrack"],
    "Vishal Mishra": ["Bollywood", "Pop"],
    "Shilpa Rao": ["Bollywood", "Pop"],
    "Vishal-Shekhar": ["Bollywood", "Soundtrack"],
    "Diljit Dosanjh": ["Punjabi", "Pop"],
    "Juss": ["Pop"],
    "yung kai": ["Rap", "Hip-Hop"],
    "Mohammad Faiz": ["Bollywood", "Classical"],
    "Amitabh Bhattacharya": ["Bollywood", "Pop"],
    "Sachet Tandon": ["Bollywood", "Pop"],
    "AUR": ["Pop"],
    "Khushi TDT": ["Pop"],
    "Hariharan": ["Bollywood", "Classical"],
    "Atif Aslam": ["Bollywood", "Pop"],
    "Mitraz": ["Pop"],
    "LA NUEVA ESCUELA": ["Reggaeton"],
    "El Reja": ["Reggaeton"],
    "Engel Montaz": ["Reggaeton"],
    "Flamen Beretta": ["Reggaeton"],
    "La Deskarga": ["Reggaeton"],
    "Bauti Mascia": ["Reggaeton"],
    "Max Carra": ["Reggaeton"],
    "Mrs. GREEN APPLE": ["J-Pop", "Alternative Rock"],
    "Da-iCE": ["J-Pop"],
    "Vaundy": ["J-Pop"],
    "Creepy Nuts": ["Rap", "Hip-Hop"],
    "Omoinotake": ["J-Pop"],
    "tuki.": ["J-Pop"],
    "Kocchi no Kento": ["J-Pop"],
    "AKASAKI": ["J-Pop"],
    "back number": ["J-Pop"],
    "King Gnu": ["J-Pop", "Rock"],
    "Kenshi Yonezu": ["J-Pop"],
    "CUTIE STREET": ["J-Pop"],
    "Number_i": ["J-Pop"],
    "Fujii Kaze": ["J-Pop"],
    "ヨルシカ": ["J-Pop"],
    "MISAMO": ["K-Pop"],
    "OFFICIAL HIGE DANDISM": ["J-Pop"],
    "YOASOBI": ["J-Pop"],
    "Yuuri": ["J-Pop"],
    "Fontaines D.C.": ["Indie Rock"],
    "Pitbull": ["Reggaeton", "Pop"],
    "Lil Naay": ["Rap", "Hip-Hop"],
    "Keyviem": ["Pop", "Rap"],
    "Shadow Blow": ["Rap", "Hip-Hop"],
    "El Blachy": ["Rap", "Hip-Hop"],
    "Xavi The Destroyer": ["Rap", "Hip-Hop"],
    "Braulio Fogon": ["Reggaeton"],
    "Amenazzy": ["Reggaeton"],
    "Jezzy": ["Rap", "Hip-Hop"],
    "NTG": ["Rap", "Hip-Hop"],
    "La Perversa": ["Reggaeton"],
    "Aventura": ["Bachata"],
    "Sin Nombre": ["Reggaeton"],
    "Romeo Santos": ["Bachata", "Pop"],
    "Dionela": ["Reggaeton"],
    "Maki": ["Pop"],
    "Arthur Nery": ["Pop", "R&B"],
    "Up Dharma Down": ["Indie Rock"],
    "Cup of Joe": ["Pop"],
    "PARTYNEXTDOOR": ["R&B", "Hip-Hop"],
    "NOBITA": ["Pop", "Indie"],
    "Hale": ["Rock", "Indie"],
    "Robledo Timido": ["Pop"],
    "Adie": ["Pop"],
    "Over October": ["Pop", "Indie"],
    "The Script": ["Pop", "Rock"],
    "HELLMERRY": ["Pop", "Rock"],
    "Bruno Mars": ["Pop", "R&B"],
    "ALLMO$T": ["Rap", "Hip-Hop"],
    "Daniel Caesar": ["R&B"],
    "Ariana Grande": ["Pop", "R&B"],
    "Orange & Lemons": ["Pop", "Indie"],
    "TJ Monterde": ["Pop", "OPM"],
    "Calein": ["Pop"],
    "gins&melodies": ["Pop"],
    "December Avenue": ["Pop", "OPM"],
    "Arctic Monkeys": ["Rock", "Indie"],
    "Flow G": ["Rap", "Hip-Hop"],
    "Hev Abi": ["Pop"],
    "Andmesh": ["Pop", "Indie"],
    "For Revenge": ["Pop"],
    ".Feast": ["Electronic"],
    "The Lantis": ["Pop"],
    "Ernie Zakri": ["Pop"],
    "Rio Clappy": ["Pop"],
    "Bernadya": ["Pop"],
    "Lyodra": ["Pop"],
    "Last Child": ["Pop"],
    "Anggi Marito": ["Pop"],
    "Juicy Luicy": ["Pop"],
    "Dudy Oris": ["Pop"],
    "Rony Parulian": ["Pop"],
    "Hindia": ["Pop", "Indie"],
    "Adrian Khalif": ["Pop"],
    "Vierra": ["Pop", "Indie"],
    "Nadin Amizah": ["Pop", "Indie"],
    "Feby Putri": ["Pop"],
    "Mahalini": ["Pop", "Indie"],
    "Nyoman Paul": ["Pop", "Indie"],
    "Virgoun": ["Pop", "Indie"],
    "Hal": ["Pop"],
    "Batas Senja": ["Pop", "Indie"],
    "Tulus": ["Pop", "Indie"],
    "Nidji": ["Pop", "Indie"],
    "Suara Kayu": ["Pop", "Indie"],
    "Yovie & Nuno": ["Pop", "Indie"],
    "Nadhif Basalamah": ["Pop"],
    "Meiska": ["Pop"],
    "NDX A.K.A.": ["Pop"],
    "Raim Laode": ["Pop"],
    "Betrand Putra Onsu": ["Pop"],
    "Tiara Andini": ["Pop"],
    "RIAN": ["Pop"],
    "Djo": ["Pop"],
    "Chase Atlantic": ["Pop", "Indie"],
    "La Única Tropical": ["Salsa"],
    "Pascal": ["Pop"],
    "Trueno": ["Rap", "Hip-Hop"],
    "Milo j": ["Rap", "Hip-Hop"],
    "Vance Joy": ["Indie", "Pop"],
    "Luke Combs": ["Country"],
    "Taylor Swift": ["Pop", "Country"],
    "Royel Otis": ["Pop", "Indie"],
    "The Kid LAROI": ["Pop", "Rap"],
    "The Goo Goo Dolls": ["Pop Rock"],
    "Xolidayboy": ["Trap"]
}

In [ ]:
# Calculate weights in the network
country_genre_weights = defaultdict(lambda: defaultdict(float))

# Add edges with weights
for index, row in spotify_global_data.iterrows():
    node_a = row.iloc[0]  # First column (country)
    for node_b in artist_nodes:
        if pd.notna(row[node_b]):
            generes = dictionary_generes[node_b]
            # Equal weight for each genre the artist belongs to
            genre_weight = 1 / len(generes)
            # Add weight for each genre in this country
            for genre in generes:
                country_genre_weights[node_a][genre] += genre_weight

# Create graph
G_genere = nx.Graph()
for country, genres in country_genre_weights.items():
    G_genere.add_node(country, bipartite=0)  # Country node
    for genre, weight in genres.items():
        G_genere.add_node(genre, bipartite=1)  # Genre node
        G_genere.add_edge(country, genre, weight=weight)

Now we will display the graph obtained.

In [ ]:
# Generate force-directed layout for genere nodes
country_nodes = [node for node, data in G_genere.nodes(data=True) if data.get("bipartite") == 0]
country_labels = {node: node for node in country_nodes}
genere_nodes = [node for node, data in G_genere.nodes(data=True) if data.get("bipartite") == 1]
genere_labels = {node: node for node in genere_nodes}

# Ensure that all country nodes have positions
fixed_positions = {node: pos for node, pos in country_positions.items() if node in country_nodes}
for node in country_nodes:
    if node not in fixed_positions:
        fixed_positions[node] = np.random.rand(2)  # Random position for missing nodes

# Spring layout for genere nodes, with country nodes fixed
positions = nx.spring_layout(G_genere, pos=fixed_positions, fixed=country_nodes, weight="weight", iterations=50, k=5)

# Apply repulsion to avoid overlapping nodes
genere_positions = {node: positions[node] for node in genere_nodes}
genere_positions = repel_nodes(genere_positions, min_dist=10)

# Combine position keeping country fixed
for node in country_nodes:
    positions[node] = fixed_positions[node]
for node in genere_nodes:
    positions[node] = genere_positions[node]

# Separate positions for country and artist nodes
country_positions = {node: positions[node] for node in country_nodes}
genere_positions = {node: positions[node] for node in genere_nodes}

# Plotting
plt.figure(figsize=(14, 12))

# Draw nodes
nx.draw_networkx_nodes(G_genere, positions, nodelist=genere_nodes, node_size=150, node_color="lightblue", label="Genere")
nx.draw_networkx_nodes(G_genere, positions, nodelist=country_nodes, node_size=250, node_color="plum", label="Countries")

# Draw labels
nx.draw_networkx_labels(G_genere, country_positions, labels=country_labels, font_size=8, font_color="black")
nx.draw_networkx_labels(G_genere, positions, labels=genere_labels, font_size=8, font_color="black")

# Draw edges
edge_widths = [((data.get("weight", 1) - 1) / 531) * (10) + 1 for _, _, data in G_genere.edges(data=True)] # Edge widths based on weight
nx.draw_networkx_edges(G_genere, positions, edge_color="gray", alpha=0.3, width=edge_widths)

# Display legend and plot
plt.legend()
plt.gca().set_frame_on(False)
plt.show()

Once the network is created, in order to determine the diversity of generes, the degree of the different countries is computed. A higher degree indicates that the country is linked to more generes, and so, it has a higher genere diversity.

In [ ]:
# Compute degrees for countries
countries = [node for node, data in G_genere.nodes(data=True) if data.get("bipartite") == 0]
country_degrees_notordered = {country: G_genere.degree(country) for country in countries}
country_degrees = {k: country_degrees_notordered[k] for k in sorted(country_degrees_notordered, key=lambda x: country_degrees_notordered[x], reverse=True)}


# Display results
print("Country Degree (Number of Connected Genres):")
for country, degree in country_degrees.items():
    print(f"{country}: {degree}")

In orter to see graphycally the results, we will make a bar chart with the results obtained.

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(country_degrees.keys(), country_degrees.values(), color="skyblue")
plt.title("Country Genre Diversity (Degree in Bipartite Network)", fontsize=14)
plt.xlabel("Country", fontsize=10)
plt.ylabel("Number of Connected Genres", fontsize=12)
plt.xticks(rotation=45, fontsize=8)
plt.tight_layout()
plt.show()

It can be seen that different countries clearly seem to have very differnt music genere diversity, this fact can be explained to different factors. First of all, countries with higher music diversity tend to have more immigartion, which enhances the consumption of different music styles. Additionally, globalization helps to get easier to different types of music and consum it.